## Pyspark

In [1]:
from pyspark.sql import SparkSession
import findspark
findspark.init()

Spark = SparkSession.builder.master("local[*]").appname("SparkByExamples.com").getOrCreate()

AttributeError: 'Builder' object has no attribute 'appname'

## Pytorch

In [4]:
import torch
torch.cuda.is_available()


if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
    
print("using", device, "device")

using cuda device


In [5]:
import time

matrix_size = 32*512

x = torch.randn(matrix_size, matrix_size)
y = torch.randn(matrix_size, matrix_size)

print("************ CPU SPEED ***************")
start = time.time()
result = torch.matmul(x,y)
print(time.time() - start)
print("verify device:", result.device)

x_gpu = x.to(device)
y_gpu = y.to(device)
torch.cuda.synchronize()

for i in range(3):
    print("************ GPU SPEED ***************")
    start = time.time()
    result_gpu = torch.matmul(x_gpu,y_gpu)
    torch.cuda.synchronize()
    print(time.time() - start)
    print("verify device:", result_gpu.device)


************ CPU SPEED ***************
33.57078170776367
verify device: cpu
************ GPU SPEED ***************
1.1165997982025146
verify device: cuda:0
************ GPU SPEED ***************
1.1169147491455078
verify device: cuda:0
************ GPU SPEED ***************
1.1077895164489746
verify device: cuda:0


## Tensorflow 

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
print("TensorFLow version:",tf.__version__)
print("Num GPUs Available:", len(tf.config.experimental.list_physical_devices('GPU')))
print("Num CPUs Available:", len(tf.config.experimental.list_physical_devices('CPU')))



TensorFLow version: 2.10.1
Num GPUs Available: 1
Num CPUs Available: 1


In [2]:
from tensorflow.python.client import device_lib
def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]
print(get_available_devices()) 

['/device:CPU:0', '/device:GPU:0']


In [3]:
tf.test.is_built_with_cuda()

True